## Ch. 02- Programming Probabilistically

In [ ]:
# Import pymc and related code
import arviz as az
import pymc as pm
import preliz as pz

In [ ]:
# Import other "data science libraries"
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 2.1 Probabilistic programming

#### 2.1.1 Flipping coins the PyMC way2.1.1 Flipping coins the PyMC way

In [ ]:
# Initialize repeatable random number generator
rng = np.random.default_rng(123)

In [ ]:
# Generate "fake real data"
trials = 4
theta_real = 0.35 # unknown in a real experiment
data = pz.Binomial(
    n=1,
    p=theta_real).rvs(trials,
                      random_state=rng.integers(np.iinfo(np.int32).max))

In [ ]:
plt.scatter(range(trials), data)
plt.show()

In [ ]:
with pm.Model() as our_first_model:
    θ = pm.Beta('θ', alpha=1., beta=1.)
    y = pm.Bernoulli('y', p=θ, observed=data)
    idata = pm.sample(1000)

### 2.2 Summarizing the posterior

In [ ]:
az.plot_trace(idata)

In [ ]:
az.plot_trace(idata, kind='rank_bars', combined=True)

In [ ]:
az.plot_posterior(idata)

### 2.3 Posterior-based decisions

### 2.3.1 Savage-Dickey density ration

In [ ]:
az.plot_bf(idata, var_name='θ', prior=rng.uniform(0, 1, 10000), ref_val=0.5)

#### 2.3.2 Region of Practical Equivalence

In [ ]:
az.plot_posterior(idata, rope=[0.45, 0.55])

In [ ]:
az.plot_posterior(idata, ref_val=0.5)

#### 2.3.3 Loss functions

In [ ]:
# Plot the loss
# The plotting part of this code is from 
# [the chapter 02 code](https://github.com/aloctavodia/BAP3/blob/main/code/Chp_02.ipynb).
grid = np.linspace(0, 1, 200)
θ_pos = idata.posterior['θ']
lossf_a = [np.mean(abs(i - θ_pos)) for i in grid]
lossf_b = [np.mean((i - θ_pos) ** 2) for i in grid]

_, ax = plt.subplots(figsize=(12, 3))
for lossf, c in zip([lossf_a, lossf_b], ['C0', 'C1']):
    mini = np.argmin(lossf)
    ax.plot(grid, lossf, c)
    ax.plot(grid[mini], lossf[mini], 'o', color=c)
    ax.annotate('{:.2f}'.format(grid[mini]),
                (grid[mini], lossf[mini] + 0.03),
                color=c)

    ax.set_yticks([])
    ax.set_xlabel(r'$\hat \theta$')

plt.show()

In [ ]:
# A (silly) assymetric loss function
lossf = []
for i in grid:
    if i < 0.5:
        f = 1 / np.median(θ_pos / np.abs(i**2 - θ_pos))
    else:
        f = np.mean((i - θ_pos) ** 2 + np.exp(-i)) - 0.25

    lossf.append(f)

In [ ]:
# Plot the (silly) assymetric loss function
mini = np.argmin(lossf)
_, ax = plt.subplots(figsize=(12, 3))
ax.plot(grid, lossf)
ax.plot(grid[mini], lossf[mini], 'o')
ax.annotate('{:.2f}'.format(grid[mini]),
(grid[mini] + 0.01, lossf[mini] + 0.1))
ax.set_yticks([])
ax.set_xlabel(r'$\hat \theta$')